In [1]:
import sys
sys.path.append("..") # Adds higher directory to python modules path.
from notebookFunctions import plotEpochAccuracyAndLoss, plotBatchAccuracyAndLoss, dirSize
from callbacks import WriteBatchResultsToCSV
from keras.callbacks import ModelCheckpoint
from generators import getBatchGenerators
from models import convertModel

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Common Variables

In [2]:
# path to data on aws
data_path = "/home/ubuntu/project/data/groundcover2016/"
model_path = "/home/ubuntu/project/model_checkpoints/"

dirs = ["maize","maizevariety","mungbean","wheat"]

# get length of the training and validation sets
maize_train_size = dirSize(data_path + "train","maize")
maize_validate_size = dirSize(data_path + "validate","maize")
train_size = dirSize(data_path + "train",dirs)
validate_size = dirSize(data_path + "validate",dirs)

shape = (256,256)
input_shape = (256,256,3)

### Convert Single class model and do more training

In [3]:
# variables
epochs = 5
batch_size = 256
steps_per_epoch = round(maize_train_size/batch_size)
validation_steps = round(maize_validate_size/batch_size)

oldModel = "../../../model_checkpoints/unet/small_maize_model.hdf5"

modelParams = {
    'n_classes' : 2,
    'dropout' : 0,
    'conv_depth' : 4,
    'input_shape' : (256,256,3)
}

sgdParams = {
    'lr' : 1.0*(10**-7),
    'weight_decay' : 1.0*(10**-10),
    'momentum' : 0.9
}

model = convertModel(oldModel,modelParams,sgdParams,'regression')

# get batch generators

trainGenerator,validateGenerator = getBatchGenerators(batch_size,data_path,shape,className = "maize",networkType = "regression")

# define callbacks 
model_checkpoint = ModelCheckpoint(model_path + 'unet/small_maize_regression_model.hdf5', monitor='loss',verbose=1, save_best_only=True)
logger = WriteBatchResultsToCSV(model_path + "unet/small_singleclass_regression_batch.csv",model_path + "unet/small_singleclass_regression_epoch.csv",5)

AAAA


In [ ]:
# fit the model
model.fit_generator(generator = trainGenerator,
                    validation_data = validateGenerator,
                    steps_per_epoch = steps_per_epoch,
                    validation_steps = validation_steps,
                    epochs = epochs,
                    use_multiprocessing = True,
                    callbacks =[model_checkpoint, logger])

Epoch 1/5
125/278 [============>.................] - ETA: 7:08 - loss: 2144686843.9040 - acc: 0.8418